<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#协程及其执行过程" data-toc-modified-id="协程及其执行过程-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>协程及其执行过程</a></span></li><li><span><a href="#例子:-用协程实现-Running-Average" data-toc-modified-id="例子:-用协程实现-Running-Average-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>例子: 用协程实现 Running Average</a></span></li><li><span><a href="#用装饰器激活协程函数" data-toc-modified-id="用装饰器激活协程函数-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>用装饰器激活协程函数</a></span></li><li><span><a href="#协程的异常处理" data-toc-modified-id="协程的异常处理-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>协程的异常处理</a></span></li><li><span><a href="#协程的返回值" data-toc-modified-id="协程的返回值-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>协程的返回值</a></span></li><li><span><a href="#yield-from" data-toc-modified-id="yield-from-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>yield from</a></span></li></ul></div>

###### 协程及其执行过程

在生成器协程中:

```python
SEND = yield RECEIVE
```

它就成了协程.
注意:

1. The value of `SEND` will only be set when the coroutine is activated later by the client code.
2. 协程退出时会抛出StopIteration异常

In [3]:
def simple_coroutine():
    print('开始协程')
    x = yield 3
    print('协程中止')


my_cro = simple_coroutine()
#激活生成函数, 函数运行到 `x = yield 3` 处阻塞
my_cro.send(None)
#给生成器发送值到x, 并且接受 生成器 yield的值
try:
    print('从协程中得到, ', my_cro.send(42))
except StopIteration as e:
    pass

<center>协程的执行过程</center>
    
![协程流程](img/16_1.png)

In [12]:
def simple_coro2(a):
    print('协程开始: a=', a)
    b = yield a
    print('协程接受: b=', b)
    c = yield a + b
    print('协程接受: c=', c)


my_coro2 = simple_coro2(14)

print('协程返回', my_coro2.send(None))

print('协程返回', my_coro2.send(28))

try:
    my_coro2.send(99)
except StopIteration as e:
    pass

协程开始: a= 14
协程返回 14
协程接受: b= 28
协程返回 42
协程接受: c= 99


###### 例子: 用协程实现 Running Average

In [19]:
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        try:
            term = yield average
        except GeneratorExit:
            break
        else:
            total += term
            count += 1
            average = total/count

In [20]:
coro_avg = averager()
item_list = [None, 1, 2, 3, 4, 5, 6, 7, 8, 9]
for i, item in enumerate(item_list):
    print(f'第{i+1}次协程的返回值: {coro_avg.send(item)}')
print(f'协程关闭时的返回值: {coro_avg.close()}')

第1次协程的返回值: None
第2次协程的返回值: 1.0
第3次协程的返回值: 1.5
第4次协程的返回值: 2.0
第5次协程的返回值: 2.5
第6次协程的返回值: 3.0
第7次协程的返回值: 3.5
第8次协程的返回值: 4.0
第9次协程的返回值: 4.5
第10次协程的返回值: 5.0
协程关闭时的返回值: None


###### 用装饰器激活协程函数
用装饰器完成协程的激活

In [21]:
from functools import wraps

def coroutine(func):
    """自己实现的装饰器函数, 完成协程函数的激活
    """
    @wraps(func)
    def primer(*args, **kwargs):
        gen = func(*args, **kwargs)
        gen.send(None)
        return gen
    return primer

###### 协程的异常处理

```python
generator.throw(exc_type[, exc_value[, traceback]])
```
在协程函数挂起的yield表达式处抛出相应的异常.

1. 如果生成器处理了这个异常, 那么执行流推进到下一个yield语句中, 且yield生成的值就作为generator.throw的返回值
2. 如果生成器没有处理异常, 异常向上冒泡, 生成器关闭

```python
generator.close()
```
在协程函数挂起的yield表达式处抛出**GeneratorExit**异常.

1. 如果生成器不处理**GeneratorExit**异常, 那么该异常不会报告给调用者
2. 如果生成器显式的处理了上述的异常, 那么生成器必须不再yield value, 否则会抛出RuntimeError异常. 生成器的任何其他异常都会冒泡给调用者

In [34]:
class DemoException(Exception):
    pass

def demo_exc_handling():
    print('协程开始')
    x = yield 
    print('接收到: ', x)
    try:
        ## print('如果没有发生异常, 生成的值为:\n')
        y = yield x
    except DemoException:
        print('如果没有发生异常, 生成的值为: {0}\n'
              '如果发生了异常, 生成的值为:\n'.format(x))   
    yield (x + 1)
    z = yield x


exc_coro = demo_exc_handling()

exc_coro.send(None)

exc_coro.send(233)

exc_coro.throw(DemoException)

协程开始
接收到:  233
如果没有发生异常, 生成的值为: 233
如果发生了异常, 生成的值为:



234

###### 协程的返回值

1. 为了得到协程函数返回值, 我们不能调用它的close方法; 需要传递一个自己定义好的终止符作为循环终止的flag.
2. 协程的返回值放在**StopIteration**的value属性里(协程正常退出时一定会抛出StopIteration异常给调用方).

In [38]:
GEN_END = 'end'
def averager_with_return():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average   
        #加入生成器终止的条件
        if term == GEN_END:
            return (count, average)        
        total += term
        count += 1
        average = total / count               

In [40]:
coro_avg = averager_with_return()
item_list = [None, 1, 2, 3, 4, 5, GEN_END]
for i, item in enumerate(item_list):
    try:
        print(f'第{i+1}次协程的返回值: {coro_avg.send(item)}')
    except StopIteration as e:
        print(f'协程关闭时的返回值: {e.value}')

第1次协程的返回值: None
第2次协程的返回值: 1.0
第3次协程的返回值: 1.5
第4次协程的返回值: 2.0
第5次协程的返回值: 2.5
第6次协程的返回值: 3.0
协程关闭时的返回值: (5, 3.0)


###### yield from

In [43]:
#yield 和 yield from的区别
def gen():
    yield range(1, 3)
    yield from range(1, 3)

print(list(gen()))

[range(1, 3), 1, 2]
